In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.formula.api import logit
import time
import pylab as pl
import seaborn as sns

mpl.style.use('fivethirtyeight')
%matplotlib inline

In [2]:
srv_url = 'https://raw.githubusercontent.com/DoxVader/RatsVsTrash/master/Data/311_Service_Requests.csv'
trash_url = 'https://raw.githubusercontent.com/DoxVader/RatsVsTrash/master/Data/Big_Belly_Alerts_2014.csv'

srvdf = pd.read_csv(srv_url)

trashdf = pd.read_csv(trash_url)

#srvdf.applymap(str).iloc[:,1]

trashdf.head(5)
    
#date_time = srvdf.loc[0:,'OPEN_DT']
#pattern = '%m/%d/%Y %H:%M:%S %p'
#epoch = int(time.mktime(time.strptime(date_time[1], pattern)))
#print date_time
#print epoch

,description,timestamp,fullness,collection,Latitude,Longitude
0,Atlantic & Milk,01/01/2014 12:41:00 AM,YELLOW,False,42.358701,-71.051439
1,1330 Boylston @ Jersey Street,01/01/2014 01:19:00 AM,YELLOW,False,42.344566,-71.097831
2,SE Brookline & Newbury (opp Popeyes chicken),01/01/2014 01:32:00 AM,YELLOW,False,42.348182,-71.097445
3,SE Brookline & Newbury (opp Popeyes chicken),01/01/2014 01:34:00 AM,RED,False,42.348182,-71.097445
4,Huntington and Dartmouth St (Library),01/01/2014 02:10:00 AM,GREEN,True,42.349327,-71.077018


In [3]:
def epochTime(dt):


#date_time = srvdf.loc[0:,'OPEN_DT']
    pattern = '%m/%d/%Y %H:%M:%S %p'
    epoch = int(time.mktime(time.strptime(dt, pattern)))
#print date_time
    return epoch
n_s=[]  
for x in srvdf.OPEN_DT:
    n_s.append(epochTime(x))

print n_s

[1320825242, 1394027213, 1319008880, 1338442669, 1448030239, 1432041720, 1348146687, 1407246775, 1418716200, 1345300959, 1429256252, 1326467037, 1384782300, 1361185390, 1381381977, 1347256970, 1367233787, 1375802445, 1380118815, 1323354884, 1431961969, 1331637824, 1352994913, 1373984771, 1352565855, 1407916505, 1381125775, 1451121060, 1350978742, 1394605795, 1446128940, 1401721833, 1347639831, 1405701207, 1381233258, 1371827979, 1384952878, 1422184589, 1345650589, 1406013085, 1375974270, 1444050675, 1338564654, 1343198129, 1409841881, 1337003900, 1400131201, 1338446752, 1441084980, 1443972660, 1378196641, 1379678230, 1443160080, 1433857734, 1349267910, 1434517560, 1450713660, 1334571907, 1433860020, 1312209805, 1311764998, 1445505300, 1444028903, 1374736919, 1406879279, 1410872220, 1345639955, 1379603076, 1335449858, 1436771640, 1348488790, 1344577374, 1360336166, 1342799297, 1370269641, 1441109287, 1378725060, 1376463659, 1362414334, 1347120591, 1396329415, 1312474054, 1340026186, 143

In [4]:
df_s=pd.DataFrame(n_s, columns=['epoch_s'])
df_s.head(5)

,epoch_s
0,1320825242
1,1394027213
2,1319008880
3,1338442669
4,1448030239


In [5]:
srvdf = srvdf.join([df_s])
srvdf.head(5)

,CASE_ENQUIRY_ID,OPEN_DT,TARGET_DT,CLOSED_DT,OnTime_Status,CASE_STATUS,CLOSURE_REASON,CASE_TITLE,SUBJECT,REASON,...,land_usage,LOCATION_STREET_NAME,LOCATION_ZIPCODE,Property_Type,Property_ID,LATITUDE,LONGITUDE,Source,Geocoded_Location,epoch_s
0,101000352507,11/09/2011 02:54:02 PM,12/09/2011 02:54:02 PM,01/27/2012 01:50:42 PM,OVERDUE,Closed,Case Closed VIOCOR: Violation Corrected,Rodent Activity,Inspectional Services,Environmental Services,...,R3,9 Burrell St,2119.0,Address,23986.0,42.3224,-71.0721,Constituent Call,"(42.3224, -71.0721)",1320825242
1,101001040880,03/05/2014 08:46:53 AM,04/04/2014 08:46:53 AM,04/01/2014 11:48:28 AM,ONTIME,Closed,Case Closed VIOCOR: Violation Corrected,Rodent Activity,Inspectional Services,Environmental Services,...,EA,234 Harrison Ave,2111.0,Address,247844.0,42.3477,-71.0617,Constituent Call,"(42.3477, -71.0617)",1394027213
2,101000344585,10/19/2011 03:21:20 PM,11/18/2011 03:21:20 PM,10/27/2011 01:38:54 PM,ONTIME,Closed,Case Closed NOVIO: No Violation Found/No Cause,Rodent Activity,Inspectional Services,Environmental Services,...,RC,459 W Broadway,2127.0,Address,157033.0,42.3357,-71.0464,Constituent Call,"(42.3357, -71.0464)",1319008880
3,101000430298,05/31/2012 01:37:49 PM,06/30/2012 01:37:49 PM,08/30/2012 11:52:05 AM,OVERDUE,Closed,Case Closed VIOCOR: Violation Corrected,Rodent Activity,Inspectional Services,Environmental Services,...,R3,614 Bennington St,2128.0,Address,14829.0,42.3834,-71.0168,Constituent Call,"(42.3834, -71.0168)",1338442669
4,101001639414,11/20/2015 09:37:19 AM,12/20/2015 09:37:19 AM,NaN,OVERDUE,Open,NaN,Rodent Activity,Inspectional Services,Environmental Services,...,RC,181-187 Green St,2130.0,Address,66360.0,42.3099,-71.1059,Constituent Call,"(42.3099, -71.1059)",1448030239


In [6]:
def epochTime(dt):


#date_time = srvdf.loc[0:,'OPEN_DT']
    pattern = '%m/%d/%Y %H:%M:%S %p'
    epoch = int(time.mktime(time.strptime(dt, pattern)))
#print date_time
    return epoch
n_t=[]  
for x in trashdf.timestamp:
    n_t.append(epochTime(x))

print n_t

[1388598060, 1388557140, 1388557920, 1388558040, 1388560200, 1388560200, 1388560380, 1388560560, 1388564220, 1388565000, 1388565300, 1388565540, 1388565780, 1388568360, 1388568540, 1388579640, 1388580780, 1388583000, 1388583720, 1388583960, 1388584560, 1388584740, 1388584800, 1388587860, 1388588400, 1388589180, 1388590560, 1388592120, 1388592300, 1388592420, 1388592540, 1388592900, 1388593020, 1388593140, 1388593260, 1388593260, 1388593320, 1388593380, 1388593740, 1388593980, 1388594520, 1388594640, 1388595000, 1388595180, 1388595360, 1388595420, 1388595900, 1388596740, 1388598480, 1388598540, 1388556900, 1388557260, 1388558400, 1388558520, 1388560740, 1388561400, 1388561880, 1388562180, 1388563260, 1388563560, 1388564040, 1388565300, 1388567220, 1388567460, 1388567580, 1388567700, 1388569140, 1388569200, 1388569200, 1388569500, 1388569560, 1388569740, 1388570100, 1388570340, 1388570460, 1388570580, 1388570820, 1388570820, 1388571300, 1388571720, 1388572020, 1388572020, 1388572140, 138

In [7]:
df_t=pd.DataFrame(n_t, columns=['epoch_t'])
df_t.head(5)

,epoch_t
0,1388598060
1,1388557140
2,1388557920
3,1388558040
4,1388560200


In [8]:
trashdf = trashdf.join([df_t])
trashdf.head(5)

,description,timestamp,fullness,collection,Latitude,Longitude,epoch_t
0,Atlantic & Milk,01/01/2014 12:41:00 AM,YELLOW,False,42.358701,-71.051439,1388598060
1,1330 Boylston @ Jersey Street,01/01/2014 01:19:00 AM,YELLOW,False,42.344566,-71.097831,1388557140
2,SE Brookline & Newbury (opp Popeyes chicken),01/01/2014 01:32:00 AM,YELLOW,False,42.348182,-71.097445,1388557920
3,SE Brookline & Newbury (opp Popeyes chicken),01/01/2014 01:34:00 AM,RED,False,42.348182,-71.097445,1388558040
4,Huntington and Dartmouth St (Library),01/01/2014 02:10:00 AM,GREEN,True,42.349327,-71.077018,1388560200


In [9]:
trashdf.count()

description    51440
timestamp      51440
fullness       51440
collection     51440
Latitude       51440
Longitude      51440
epoch_t        51440
dtype: int64

### Assigning min and max in trashdf 

In [10]:
min_t = trashdf.loc[0,'epoch_t']
max_t = trashdf.loc[51439,'epoch_t']

In [11]:
# Adjusting totals in srvdf
srvdf.count()

CASE_ENQUIRY_ID                   10188
OPEN_DT                           10188
TARGET_DT                         10188
CLOSED_DT                          9516
OnTime_Status                     10187
CASE_STATUS                       10188
CLOSURE_REASON                     9531
CASE_TITLE                        10188
SUBJECT                           10188
REASON                            10188
TYPE                              10188
QUEUE                             10188
Department                        10188
SubmittedPhoto                      114
ClosedPhoto                           0
Location                           9767
fire_district                      9744
pwd_district                       9761
city_council_district              9765
police_district                    9760
neighborhood                       9760
neighborhood_services_district     9765
ward                               9767
precinct                           9746
land_usage                         8992


In [12]:
srvdf = srvdf[srvdf.epoch_s > min_t]
srvdf = srvdf[srvdf.epoch_s <= max_t]

srvdf.head(5)

,CASE_ENQUIRY_ID,OPEN_DT,TARGET_DT,CLOSED_DT,OnTime_Status,CASE_STATUS,CLOSURE_REASON,CASE_TITLE,SUBJECT,REASON,...,land_usage,LOCATION_STREET_NAME,LOCATION_ZIPCODE,Property_Type,Property_ID,LATITUDE,LONGITUDE,Source,Geocoded_Location,epoch_s
1,101001040880,03/05/2014 08:46:53 AM,04/04/2014 08:46:53 AM,04/01/2014 11:48:28 AM,ONTIME,Closed,Case Closed VIOCOR: Violation Corrected,Rodent Activity,Inspectional Services,Environmental Services,...,EA,234 Harrison Ave,2111.0,Address,247844.0,42.3477,-71.0617,Constituent Call,"(42.3477, -71.0617)",1394027213
7,101001144232,08/05/2014 09:52:55 PM,09/04/2014 09:52:55 PM,08/20/2014 09:01:47 AM,ONTIME,Closed,Case Closed CLOSED: Closed/No Permit,Rodent Activity,Inspectional Services,Environmental Services,...,R3,87 Burrell St,2119.0,Address,23983.0,42.3235,-71.0685,Self Service,"(42.3235, -71.0685)",1407246775
8,101001234215,12/16/2014 02:50:00 PM,01/15/2015 02:50:00 PM,01/14/2015 10:35:53 AM,ONTIME,Closed,Case Closed NOVIO: No Violation Found/No Cause,Rodent Activity,Inspectional Services,Environmental Services,...,CM,40-50 Battery St,2109.0,Address,8699.0,42.3663,-71.0496,Constituent Call,"(42.3663, -71.0496)",1418716200
25,101001149737,08/13/2014 03:55:05 PM,09/12/2014 03:55:05 PM,09/05/2014 03:12:35 PM,ONTIME,Closed,Case Closed CORR: Corrected,Rodent Activity,Inspectional Services,Environmental Services,...,R4,32 Parker St,2129.0,Address,107766.0,42.3828,-71.0792,Self Service,"(42.3828, -71.0792)",1407916505
29,101001045864,03/12/2014 02:29:55 PM,04/11/2014 02:29:55 PM,04/17/2014 11:04:45 AM,OVERDUE,Closed,Case Closed VIOCOR: Violation Corrected,Rodent Activity,Inspectional Services,Environmental Services,...,R1,241 Leyden St,2128.0,Address,85803.0,42.3884,-71.0019,Constituent Call,"(42.3884, -71.0019)",1394605795
